In [82]:
import pandas as pd 
import numpy as np
#import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
#from sklearn.feature_selection import RFE 
#from sklearn.model_selection import train_test_split, cross_val_score 
#from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import KFold
#from sklearn.model_selection import RepeatedKFold

wdir = 'C:/jupyter_devel/kist-europe/QSAR/mmc1_data/'
csv = 'mmc1-androgen-maccs-activity.csv'

df = pd.read_csv(wdir+csv)
df = df.dropna(axis=0) 

### data load

In [83]:
validation_set = []
val_set_idx = []
for i in df.index : 
    if df.loc[i]['Set'] == 'External validation set': 
        validation_set.append(df.iloc[i])
        val_set_idx.append(i)
        
val_df = pd.DataFrame(validation_set) 
y_test = val_df['Class']
x_test = val_df.drop(['Set','Class'],axis=1)

In [84]:
train_df = df.drop(val_set_idx)
y_train = train_df['Class']
x_train = train_df.drop(['Set','Class'],axis=1)

### model setting

#### knn classifier

In [124]:
neigh = KNeighborsClassifier(n_neighbors=4,weights='uniform',metric='jaccard')
neigh.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [125]:
metrics.accuracy_score(y_test,neigh.predict(x_test))

0.6371681415929203

In [126]:
fpr,tpr,thresholds = metrics.roc_curve(y_test,neigh.predict(x_test),pos_label=1)
metrics.auc(fpr,tpr)

0.5636212624584718

#### support vector classifier

In [98]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

svc = SVC(kernel='linear',gamma='auto')

feature = range(10,30,2)

for f in feature : 
    print('feature : {}'.format(f))
    rfe = RFE(estimator=svc,n_features_to_select=f,step=1)
    rfe.fit(x_train,y_train)
    
    fpr,tpr,thresholds = metrics.roc_curve(y_test,rfe.predict(x_test),pos_label=1)
    print('ACC : {}  | AUC : {}'.format(metrics.accuracy_score(y_test,rfe.predict(x_test)),metrics.auc(fpr,tpr) ))
    
#best N of feature on this model
#feature : 18
#ACC : 0.6106194690265486  | AUC : 0.515282392026578

feature : 10
ACC : 0.6017699115044248  | AUC : 0.4857142857142857
feature : 12
ACC : 0.6017699115044248  | AUC : 0.508139534883721
feature : 14
ACC : 0.6017699115044248  | AUC : 0.5036544850498338
feature : 16
ACC : 0.6017699115044248  | AUC : 0.4991694352159468
feature : 18
ACC : 0.6106194690265486  | AUC : 0.515282392026578
feature : 20
ACC : 0.6106194690265486  | AUC : 0.510797342192691
feature : 22
ACC : 0.6106194690265486  | AUC : 0.510797342192691
feature : 24
ACC : 0.6017699115044248  | AUC : 0.5036544850498338
feature : 26
ACC : 0.5929203539823009  | AUC : 0.4965116279069767
feature : 28
ACC : 0.5929203539823009  | AUC : 0.49202657807308964


In [107]:
from sklearn.svm import LinearSVC 

linear_svc = LinearSVC()
linear_svc.fit(x_train,y_train)

fpr,tpr,thresholds = metrics.roc_curve(y_test,linear_svc.predict(x_test),pos_label=1)
print('ACC : {}  | AUC : {}'.format(metrics.accuracy_score(y_test,linear_svc.predict(x_test)),metrics.auc(fpr,tpr) ))



ACC : 0.6017699115044248  | AUC : 0.521594684385382


C:\Users\chon0\Anaconda3\envs\develroom\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
